In [4]:
import pandas as pd
import json
import numpy as np


In [ ]:

def safe_get(data, keys, default=None):
    """Safely get nested dictionary values"""
    current = data
    for key in keys:
        if isinstance(current, dict) and key in current:
            current = current[key]
        else:
            return default
    return current

def update_main(df2):

    consolidate = pd.read_csv('../cleaned/localidades.csv')
    consolidate = consolidate.set_index('id')
    # Perform left join
    # merged_df = pd.merge(consolidate, df2, on=['id','rent_size_1_bedroom','rent_size_2_bedroom','rent_size_3_bedroom','rent_size_4_bedroom','rent_size_5_bedroom'], how='left')
    merged_df =   consolidate.combine_first(df2)  
    # Reset index
    # merged_df = merged_df.reset_index(drop=True)
    
    merged_df.to_csv('../cleaned/localidades.csv')
# Extract buckets data
def parse_buckets_json(localidad_id):
    """
    Parse JSON file with bucket data and create a DataFrame with bucket_min as columns
    
    Args:
        file_path (str): Path to the JSON file
    
    Returns:
        pandas.DataFrame: DataFrame with id and bucket values as columns
    """
    # Read JSON from file

    with open(f"../sources/{localidad_id}/listing_type.json", 'r') as file:
        data = json.load(file)
    
    # Extract id and buckets using safe_get
    submarket_id = safe_get(data, ['payload', 'submarket_id'])
    buckets = safe_get(data, ['payload', 'buckets'], [])
        
        # If no buckets found, return empty DataFrame
    if not buckets:
        print("No buckets found in the JSON data.")
        return pd.DataFrame()
        
        # Create initial dictionary with id
    row_data = {'id': int(submarket_id)}
        
        # Add each bucket value to the row
    for bucket in buckets:
        category = safe_get(bucket, ['category'])
        value = safe_get(bucket, ['value'])
            
        column_name = f'rent_{category}'
        row_data[column_name] = value
        
        # Create DataFrame from the single row
    df = pd.DataFrame([row_data])
    df = df.set_index('id')
    updated = update_main(df)
        
    return updated

In [5]:
if __name__ == "__main__":
    # Specify your JSON file path here
    localidad_id="142649"
    # Parse the JSON file
    df = parse_buckets_json(localidad_id)
    print(df)


None
